In [ ]:
import pickle
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import tkinter as tk
from tkinter import ttk

class AQIForecastGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Pollutant Forecasting and Suggestions GUI Mixed")

        # Defining pollutant names, model filenames, and units
        pollutants = ['AQI', 'CO', 'NO2', 'PM10', 'PM2.5', 'O3','SO2']
        model_filenames = [
            'Nishanthsarima4_model.pkl',
            'Nishanthsarima_model_CO.pkl',
            'Nishanthsarima_model_NO2.pkl',
            'Nishanthsarima_model_pm10.pkl',
            'Nishanthsarima_model1_pm2_5.pkl',
            'Nishanthsarima_model_SO2.pkl',
            'Nishanthsarima_model_O3.pkl'
        ]
        units = ['-', 'ppm', 'ppb', 'µg/m³', 'µg/m³', 'ppb', 'ppb']

        # Creating a dictionary to store loaded models and units
        self.loaded_models = {pollutant: {'model': None, 'unit': unit, 'filename': filename} for pollutant, filename, unit in zip(pollutants, model_filenames, units)}

        # Loading of saved SARIMA models for each pollutant
        for pollutant, model_info in self.loaded_models.items():
            with open(model_info['filename'], 'rb') as file:
                model_info['model'] = pickle.load(file)

        # Extension of time index to include the forecast period
        self.extended_index = pd.date_range(start='2015-01-01', end='2025-12-31', freq='D')

        # GUI components
        self.label_date = ttk.Label(root, text="Enter Date (YYYY-MM-DD):")
        self.label_date.pack(pady=10)

        self.date_entry = ttk.Entry(root)
        self.date_entry.pack(pady=10)

        self.forecast_button = ttk.Button(root, text="Get Forecast and Suggestions", command=self.get_forecast_and_suggestions)
        self.forecast_button.pack(pady=10)

        self.result_label = ttk.Label(root, text="")
        self.result_label.pack(pady=10)

    def get_forecast_and_suggestions(self):
        desired_date = self.date_entry.get()

        try:
            pd.to_datetime(desired_date)
        except ValueError:
            self.result_label.config(text="Invalid date format. Please use YYYY-MM-DD.")
            return

        if pd.to_datetime(desired_date) not in self.extended_index:
            self.result_label.config(text="'desired_date' should be within the range of the extended index.")
            return

        # Initialize result text
        result_text = ""

        # Make predictions for each pollutant using the loaded models
        for pollutant, model_info in self.loaded_models.items():
            forecast = model_info['model'].get_prediction(start=desired_date, end=desired_date, dynamic=False)
            predicted_value = forecast.predicted_mean.iloc[0]

            # Categorization and suggestions for each pollutant
            if pollutant == 'AQI':
                res = int(predicted_value)
                if res <= 50:
                    suggestion = "Enjoy the fresh air! Air quality is good with minimal impact."
                elif 51 <= res <= 100:
                    suggestion = "Air quality is satisfactory. Minor breathing discomfort to sensitive people."
                elif 101 <= res <= 200:
                    suggestion = "Air quality is moderate. Breathing discomfort to people with lungs, asthma, and heart diseases."
                elif 201 <= res <= 400:
                    suggestion = "Air quality is very poor. Breathing discomfort to most people on prolonged exposure."
                elif 401 <= res <= 500:
                    suggestion = "Air quality is severe. Affects healthy people and seriously impacts those with existing diseases."

                result_text += f"\nThe forecasted {pollutant} value for {desired_date} is: {predicted_value} {model_info['unit']}\n"
                result_text += f"Suggestion: {suggestion}\n"
                result_text += f"And the pollutants as below: \n"
            else:
                result_text += f"\n {pollutant} level = {predicted_value} {model_info['unit']}\n"
            
        # Categorization and suggestions for each pollutant
        for pollutant, model_info in self.loaded_models.items():
            forecast = model_info['model'].get_prediction(start=desired_date, end=desired_date, dynamic=False)
            predicted_value = forecast.predicted_mean.iloc[0]
            
            # Categorization and suggestions for each pollutant
            if pollutant == 'AQI':
                #modifying the predicted value
                #predicted_value = (predicted_value + 2) / 2
                res = int(predicted_value)
                if res <= 50:
                    suggestion = "Enjoy the fresh air! Air quality is good with minimal impact."
                elif 51 <= res <= 100:
                    suggestion = "Air quality is satisfactory. Minor breathing discomfort to sensitive people."
                elif 101 <= res <= 200:
                    suggestion = "Air quality is moderate. Breathing discomfort to people with lungs, asthma, and heart diseases."
                elif 201 <= res <= 400:
                    suggestion = "Air quality is very poor. Breathing discomfort to most people on prolonged exposure."
                elif 401 <= res <= 500:
                    suggestion = "Air quality is severe. Affects healthy people and seriously impacts those with existing diseases."

                result_text += f"The new pollutant values based on powerplant being added: \n"
                #result_text += f"\n NEW {pollutant} : {predicted_value} {model_info['unit']}\n"
                #result_text += f"Suggestion: {suggestion}\n"
                #result_text += f"tThe new pollutant values based on powerplant being added: \n"
            elif pollutant == 'CO':
                #modifying the predicted value
                predicted_value = (predicted_value + 6)
                result_text += f"\n {pollutant} level = {predicted_value} {model_info['unit']}\n"
            elif pollutant == 'NO2':
                #modifying the predicted value
                predicted_value = (predicted_value + ((40/365) * (1-0.51)))
                result_text += f"\n {pollutant} level = {predicted_value} {model_info['unit']}\n"
            elif pollutant == 'PM10':
                #modifying the predicted value
                predicted_value = (predicted_value + ((50) * (1-0.56)))
                result_text += f"\n {pollutant} level = {predicted_value} {model_info['unit']}\n"
            elif pollutant == 'PM2.5':
                #modifying the predicted value
                predicted_value = (predicted_value + (25/365) * (1-0.56))
                result_text += f"\n {pollutant} level = {predicted_value} {model_info['unit']}\n"
            elif pollutant == 'O3':
                #modifying the predicted value
                predicted_value = (predicted_value + (120/(3*365)))
                result_text += f"\n {pollutant} level = {predicted_value} {model_info['unit']}\n"
            elif pollutant == 'SO2':
                #modifying the predicted value
                predicted_value = (predicted_value + (125 * (1-0.66)))
                result_text += f"\n {pollutant} level = {predicted_value} {model_info['unit']}\n"

        
        # Displaying the forecasted values and suggestions for the specified date
        self.result_label.config(text=result_text)

if __name__ == "__main__":
    root = tk.Tk()
    app = AQIForecastGUI(root)
    root.mainloop()